In [ ]:
import sys
print sys.version

In [ ]:
%pylab inline

путь к тестовому файлу ожидаю здесь. Ожидаю что там будет аналогчный трейну CSV

In [ ]:

testFilePath = 'X_train-0.csv'

In [ ]:
from gensim import utils
from gensim.models import Word2Vec

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDRegressor
from sklearn.svm import LinearSVC, LinearSVR

from catboost import Pool, CatBoostRegressor

from collections import defaultdict
from operator import itemgetter
import time
import cPickle as pickle

from pymystem3 import Mystem
import re

mystem = Mystem()
modifiers = "\xcc\x81|<OTHER>|<MODIFIERS>"

random_seed = int(time.time())
print random_seed


### поднятие модели

да, некрасиво, но когда времени по часу вечером не до красоты

In [ ]:
!wget https://www.dropbox.com/s/k365es9eghn6ep8/MODEL.pcl?dl=0 > MODEL.pcl

In [ ]:
with open('MODEL.pcl', ) as dumpFile:
    combinedModel = pickle.load(dumpFile)

In [ ]:
defaultModel = combinedModel[0]
positiveModel = combinedModel[1]
negativeModel = combinedModel[2]
commonModel = combinedModel[3]

In [ ]:

defaultMeanLR = combinedModel[4]
defaultNormalizedLR = combinedModel[5]
positiveMeanLR = combinedModel[6]
positiveNormalizedLR = combinedModel[7]
negativeMeanLR = combinedModel[8]
negativeNormalizedLR = combinedModel[9]
commonMeanLR = combinedModel[10]
commonNormalizedLR = combinedModel[11]
concatMeanLR = combinedModel[12]
concatNormalizedLR = combinedModel[13]

In [ ]:

defaultMeanSGD = combinedModel[14]
defaultNormalizedSGD = combinedModel[15]
positiveMeanSGD = combinedModel[16]
positiveNormalizedSGD = combinedModel[17]
negativeMeanSGD = combinedModel[18]
negativeNormalizedSGD = combinedModel[19]
commonMeanSGD = combinedModel[20]
commonNormalizedSGD = combinedModel[21]
concatMeanSGD = combinedModel[22]
concatNormalizedSGD = combinedModel[23]

In [ ]:

defaultMeanSVMR = combinedModel[24]
defaultNormalizedSVMR = combinedModel[25]
positiveMeanSVMR = combinedModel[26]
positiveNormalizedSVMR = combinedModel[27]
negativeMeanSVMR = combinedModel[28]
negativeNormalizedSVMR = combinedModel[29]
commonMeanSVMR = combinedModel[30]
commonNormalizedSVMR = combinedModel[31]
concatMeanSVMR = combinedModel[32]
concatNormalizedSVMR = combinedModel[33]

In [ ]:
defaultMeanSVMC = combinedModel[34]
defaultNormalizedSVMC = combinedModel[35]
positiveMeanSVMC = combinedModel[36]
positiveNormalizedSVMC = combinedModel[37]
negativeMeanSVMC = combinedModel[38]
negativeNormalizedSVMC = combinedModel[39]
commonMeanSVMC = combinedModel[40]
commonNormalizedSVMC = combinedModel[41]
concatMeanSVMC = combinedModel[42]
concatNormalizedSVMC = combinedModel[43]

In [ ]:
mean_std_default = combinedModel[44]
mean_std_default_2 = combinedModel[45]
mean_std_u_default = combinedModel[46]
mean_std_u_default_2 = combinedModel[47]
mean_std_common = combinedModel[48]
mean_std_common_2 = combinedModel[49]
mean_std_u_common = combinedModel[50]
mean_std_u_common_2 = combinedModel[51]
mean_std_pos = combinedModel[52]
mean_std_pos_2 = combinedModel[53]
mean_std_u_pos = combinedModel[54]
mean_std_u_pos_2 = combinedModel[55]
mean_std_neg = combinedModel[56]
mean_std_neg_2 = combinedModel[57]
mean_std_u_neg = combinedModel[58]
mean_std_u_neg_2 = combinedModel[59]

In [ ]:
mean_std_default_hq = combinedModel[60]
mean_std_default_shq = combinedModel[61]
mean_std_u_default_hq = combinedModel[62]
mean_std_u_default_shq = combinedModel[63]
mean_std_common_hq = combinedModel[64]
mean_std_common_shq = combinedModel[65]
mean_std_u_common_hq = combinedModel[66]
mean_std_u_common_shq = combinedModel[67]
mean_std_pos_hq = combinedModel[68]
mean_std_pos_shq = combinedModel[69]
mean_std_u_pos_hq = combinedModel[70]
mean_std_u_pos_shq = combinedModel[71]
mean_std_neg_hq = combinedModel[72]
mean_std_neg_shq = combinedModel[73]
mean_std_u_neg_hq = combinedModel[74]
mean_std_u_neg_shq = combinedModel[75]

In [ ]:
catBoost = combinedModel[76]

# Применение модели

In [ ]:
data = pd.read_csv(testFilePath, ',')
data.head()

In [ ]:
def simplify_comment(comment):
    if comment is nan:
        return None
    lemmas = mystem.lemmatize(comment)
    lemmas = filter(lambda x: not x.isspace(), lemmas)
    if len(lemmas) == 0:
        #print comment
        return None
    ne_lemmas = []
    for i in xrange(0, len(lemmas)):
        if lemmas[i-1] == 'не':
            ne_lemmas[-1] = 'не_' + lemmas[i]
        else:
            ne_lemmas.append(lemmas[i])
    return ne_lemmas

In [ ]:
def get_vector(comment, model):
    if comment is NaN:
        return (np.zeros(100), np.zeros(100), np.zeros(100))
    simple = simplify_comment(comment)
    vector = np.zeros(100)
    count = 0;
    for word in simple:
        if word in model:
            vector += model[word]
            count += 1
    if count == 0:
        return (np.zeros(100), np.zeros(100), np.zeros(100))
    mean = vector / count
    normalized = vector / np.linalg.norm(vector)
    return vector, mean, normalized
    
    

In [ ]:
defaultVectorsTest = []
positiveVectorsTest = []
negativeVectorsTest = []
commonVectorsTest = []

defaultVectorsTestMean = []
positiveVectorsTestMean = []
negativeVectorsTestMean = []
commonVectorsTestMean = []

defaultVectorsTestNormalized = []
positiveVectorsTestNormalized = []
negativeVectorsTestNormalized = []
commonVectorsTestNormalized = []

for opinion in data.itertuples(index=True, name='Opinion'):
    comments = []
    comment = opinion.comment
    vectors = get_vector(comment, defaultModel)
    defaultVectorsTest.append(vectors[0])
    defaultVectorsTestMean.append(vectors[1])
    defaultVectorsTestNormalized.append(vectors[2])
    comments.append('' if opinion.comment is NaN else opinion.comment)
    
    comment = opinion.commentPositive
    vectors = get_vector(comment, positiveModel)
    positiveVectorsTest.append(vectors[0])
    positiveVectorsTestMean.append(vectors[1])
    positiveVectorsTestNormalized.append(vectors[2])
    comments.append('' if opinion.commentPositive is NaN else opinion.commentPositive)

    comment = opinion.commentNegative
    vectors = get_vector(comment, negativeModel)
    negativeVectorsTest.append(vectors[0])
    negativeVectorsTestMean.append(vectors[1])
    negativeVectorsTestNormalized.append(vectors[2])
    comments.append('' if opinion.commentNegative is NaN else opinion.commentNegative)

    comment = ' '.join(comments)
    vectors = get_vector(comment, commonModel)
    commonVectorsTest.append(vectors[0])
    commonVectorsTestMean.append(vectors[1])
    commonVectorsTestNormalized.append(vectors[2])

In [ ]:
defaultVectorsTest = np.array(defaultVectorsTest)
defaultVectorsTestMean = np.array(defaultVectorsTestMean)
defaultVectorsTestNormalized = np.array(defaultVectorsTestNormalized)
commonVectorsTest = np.array(commonVectorsTest)
commonVectorsTestMean = np.array(commonVectorsTestMean)
commonVectorsTestNormalized = np.array(commonVectorsTestNormalized)
positiveVectorsTest = np.array(positiveVectorsTest)
positiveVectorsTestMean = np.array(positiveVectorsTestMean)
positiveVectorsTestNormalized = np.array(positiveVectorsTestNormalized)
negativeVectorsTest = np.array(negativeVectorsTest)
negativeVectorsTestMean = np.array(negativeVectorsTestMean)
negativeVectorsTestNormalized = np.array(negativeVectorsTestNormalized)

vectorsTest = np.hstack((defaultVectorsTest, positiveVectorsTest, negativeVectorsTest, commonVectorsTest))
vectorsTestMean = np.hstack((defaultVectorsTestMean, positiveVectorsTestMean, negativeVectorsTestMean, commonVectorsTestMean))
vectorsTestNormalized = np.hstack((defaultVectorsTestNormalized, positiveVectorsTestNormalized, negativeVectorsTestNormalized, commonVectorsTestNormalized))

In [ ]:
testLM = np.vstack(( defaultMeanLR.predict(defaultVectorsTestMean),
                     defaultNormalizedLR.predict(defaultVectorsTestNormalized),
                     defaultMeanSVMC.predict(defaultVectorsTestMean),
                     defaultNormalizedSVMC.predict(defaultVectorsTestNormalized),
                     positiveMeanLR.predict(positiveVectorsTestMean),
                     positiveNormalizedLR.predict(positiveVectorsTestNormalized),
                     positiveMeanSVMC.predict(positiveVectorsTestMean),
                     positiveNormalizedSVMC.predict(positiveVectorsTestNormalized),
                     negativeMeanLR.predict(negativeVectorsTestMean),
                     negativeNormalizedLR.predict(negativeVectorsTestNormalized),
                     negativeMeanSVMC.predict(negativeVectorsTestMean),
                     negativeNormalizedSVMC.predict(negativeVectorsTestNormalized),
                     commonMeanLR.predict(commonVectorsTestMean),
                     commonNormalizedLR.predict(commonVectorsTestNormalized),
                     commonMeanSVMC.predict(commonVectorsTestMean),
                     commonNormalizedSVMC.predict(commonVectorsTestNormalized),
                     concatMeanLR.predict(vectorsTestMean),
                     concatNormalizedLR.predict(vectorsTestNormalized),
                     concatMeanSVMC.predict(vectorsTestMean),
                     concatNormalizedSVMC.predict(vectorsTestNormalized),
                     concatMeanSVMR.predict(vectorsTestMean),
                     concatNormalizedSVMR.predict(vectorsTestNormalized),
                     concatMeanSGD.predict(vectorsTestMean),
                     concatNormalizedSGD.predict(vectorsTestNormalized),
                     commonMeanSVMR.predict(commonVectorsTestMean),
                     commonNormalizedSVMR.predict(commonVectorsTestNormalized),
                     commonMeanSGD.predict(commonVectorsTestMean),
                     commonNormalizedSGD.predict(commonVectorsTestNormalized),
                     negativeMeanSVMR.predict(negativeVectorsTestMean),
                     negativeNormalizedSVMR.predict(negativeVectorsTestNormalized),
                     negativeMeanSGD.predict(negativeVectorsTestMean),
                     negativeNormalizedSGD.predict(negativeVectorsTestNormalized),
                     positiveMeanSVMR.predict(positiveVectorsTestMean),
                     positiveNormalizedSVMR.predict(positiveVectorsTestNormalized),
                     positiveMeanSGD.predict(positiveVectorsTestMean),
                     positiveNormalizedSGD.predict(positiveVectorsTestNormalized),
                     defaultMeanSVMR.predict(defaultVectorsTestMean),
                     defaultNormalizedSVMR.predict(defaultVectorsTestNormalized),
                     defaultMeanSGD.predict(defaultVectorsTestMean),
                     defaultNormalizedSGD.predict(defaultVectorsTestNormalized))).T

testLM.shape

In [ ]:
pronoun = 'я что он как этот они мы весь который она так свой вы ты такой его себя ее когда вот другой наш самый мой кто сам там какой их потом ничто каждый потому тогда здесь какой-то что-то всегда ваш никто почему поэтому свое никогда никакой некоторый твой куда кто-то как-то зачем сей туда какой-нибудь всего откуда сюда столь где-то что-нибудь почему-то некий когда-то чего отсюда чей нечто кто-нибудь вон оттуда какой-либо таков куда-то никуда каков таковой кой оттого некогда отчего нигде кое-что когда-нибудь чей-то где-нибудь такой-то что-либо всюду как-нибудь откуда-то ничуть куда-нибудь сколь тут-то этакий тот-то так-то кое-какой кое-как кое-кто зачем-то кое-где кто-либо ихний некто отчего-то каковой эдакий нибудь тогда-то чего-то когда-либо почем отовсюду ничей доселе оный ниоткуда экий чей-нибудь сям никой'.split()
preposition = 'а-ля без безо благодаря близ в вблизи ввиду вглубь вдогон вдоль взамен включая вкруг вместо вне внизу внутри внутрь во возле вокруг вопреки вослед впереди вразрез вроде вслед вследствие для для-ради до за заместо из из-за из-под изнутри изо к касательно ко кроме кругом меж между мимо на наверху навстречу над надо назад накануне наперекор наперерез наподобие напротив насупротив насчёт ниже о об обо обок около окрест окромя округ опосля от относительно ото перед передо по по-за по-над по-под поверх под подле подо подобно позади позднее помимо поперёк порядка посередине посередь после посреди посредине посредством пред предо прежде при про против путём ради с сверх сверху свыше сзади сквозь снизу со согласно спустя среди средь сродни супротив через чрез'.split()
pronoun = set(pronoun)
preposition = set(preposition)
len(pronoun), len(preposition)

In [ ]:
def lemma_filter(x):
    ux = unicode(x,"utf8")
    return ux.isalpha() \
        and x not in pronoun \
        and x not in preposition

def process_comment(comment, rate, dictionary, u_dictionary):
    if comment is nan:
        return
    lemmas = mystem.lemmatize(comment)
    lemmas = filter(lemma_filter, lemmas)
    if len(lemmas) == 0:
        #print comment
        return
    ne_lemmas = []
    for i in xrange(0, len(lemmas)):
        if lemmas[i-1] == 'не':
            ne_lemmas.append('не_' + lemmas[i])
        elif len(unicode(lemmas[i],"utf8")) > 2:
            ne_lemmas.append(lemmas[i])
            
    for lemma in ne_lemmas:
        dictionary[lemma].append(rate)
        lemma_common_rates[lemma].append(rate)
    for lemma in set(ne_lemmas):
        u_dictionary[lemma].append(rate)
        u_lemma_common_rates[lemma].append(rate)

In [ ]:
def predict_mean(comment, mean, std=defaultdict(float), default_value=-1000., unique=False):
    if comment is nan:
        return default_value
    lemmas = mystem.lemmatize(comment)
    lemmas = filter(lemma_filter, lemmas)
    if len(lemmas) == 0:
        return default_value
    ne_lemmas = []
    if lemmas[0] != 'не':
        ne_lemmas.append(lemmas[0])
    for i in xrange(1, len(lemmas)):
        if lemmas[i-1] == 'не':
            ne_lemmas.append('не_' + lemmas[i])
        elif len(unicode(lemmas[i],"utf8")) > 2:
            ne_lemmas.append(lemmas[i])
    means = []
    stds = []
    if unique:
        ne_lemmas = set(ne_lemmas)
    for lemma in ne_lemmas:
        if lemma not in mean:
            continue
        means.append(mean[lemma])
        stds.append(std[lemma])
    if len(means) == 0:
        return default_value
    lemma_scores = np.random.normal(means, stds)
    prediction = np.mean(lemma_scores)
    return prediction

In [ ]:
rates = np.array([1.0, 1.3, 1.7])
rates = np.hstack((rates, rates + 1, rates + 2, rates + 3, [5]))
def discr(prediction):
    return rates[np.argmin(np.abs(rates - prediction))]

In [ ]:
def predict_discr_with_hq_dict(comment, mean, std=defaultdict(float), unique=False):
    if comment is nan:
        return 0
    lemmas = mystem.lemmatize(comment)
    lemmas = filter(lemma_filter, lemmas)
    if len(lemmas) == 0:
        return 0
    ne_lemmas = []
    if lemmas[0] != 'не':
        ne_lemmas.append(lemmas[0])
    for i in xrange(1, len(lemmas)):
        if lemmas[i-1] == 'не':
            ne_lemmas.append('не_' + lemmas[i])
        elif len(unicode(lemmas[i],"utf8")) > 2:
            ne_lemmas.append(lemmas[i])
    means = []
    stds = []
    if unique:
        ne_lemmas = set(ne_lemmas)
    for lemma in ne_lemmas:
        if lemma not in mean:
            continue
        means.append(mean[lemma])
        stds.append(std[lemma])
    if len(means) == 0:
        return 0
    lemma_scores = np.random.normal(means, stds)
    prediction = np.mean(lemma_scores)
    return int(discr(prediction) * 10)

In [ ]:
describe_rate = {
    5: ['5 балл', 'пять балл', 'оценка 5', 'оценка пять', 'оценка  -  5', 'оценка  -  пять', 'ставить 5', 'ставить пять', 'пятерка', 'пятерочка'],
    4: ['4 балл', 'четыре балл', 'оценка 4', 'оценка четыре', 'оценка  -  4', 'оценка  -  четыре', 'ставить 4', 'ставить четыре', 'четверка', 'четверочка'],
    3: ['3 балл', 'три балл', 'оценка 3', 'оценка три', 'оценка  -  3', 'оценка  -  три', 'ставить 3', 'ставить три', ' тройка ', 'троечка'],
    2: ['2 балл', 'два балл', 'оценка 2', 'оценка два', 'оценка  -  2', 'оценка  -  два', 'ставить 2', 'ставить два', 'двойка', 'двоечка'],
    1: ['1 балл', 'оценка 1', 'оценка один', 'оценка  -  1', 'оценка  -  один', 'ставить 1', 'ставить один', 'единица'],
}

from collections import Counter

def find_description(comment):
    if comment is NaN:
        return 0
    lemmas = mystem.lemmatize(comment)
    lemmas = filter(lambda x: not x.isspace(), lemmas)
    normalized_comment = ' '.join(lemmas)
    r = []
    for rate, describes in describe_rate.iteritems():
        for describe in describes:
            if normalized_comment.find(describe) == -1:
                continue
            r.append(rate)
    if len(r) == 0:
        return 0
    return Counter(r).most_common()[0][0]

In [ ]:
def get_features(pandas_dataset, linearModelPredictions, w2v):
    assert len(pandas_dataset) == len(linearModelPredictions)
    pool = []
    lm_index = 0
    for opinion in pandas_dataset.itertuples(index=True, name='Opinion'):
        features = []

        # 0 sku
        features.append(opinion.sku)
        # 1 brandId
        features.append(opinion.brandId)
        # 2 cat1
        features.append(opinion.categoryLevel1Id)
        # 3 cat2
        features.append(opinion.categoryLevel2Id)

        # 4 comment
        if (opinion.comment is NaN) or len(opinion.comment.split()) < 5:
            features.append("NO")
        else:
            features.append("YES")
        # 5 positiveComment
        if (opinion.commentPositive is NaN) or len(opinion.commentPositive.split()) < 5:
            features.append("NO")
        else:
            features.append("YES")
        # 6 negativeComment
        if (opinion.commentNegative is NaN) or len(opinion.commentNegative.split()) < 5:
            features.append("NO")
        else:
            features.append("YES")

        # 7 description
        features.append(find_description(opinion.comment))

        # 8 defaultDict_strong
        features.append(predict_mean(opinion.comment, mean_std_default[0]))
        # 9 defaultDict_strong_unique
        features.append(predict_mean(opinion.comment, mean_std_u_default[0], unique=True))

        # 10 defaultDict2_strong
        features.append(predict_mean(opinion.comment, mean_std_default_2[0]))
        # 11 defaultDict2_strong_unique
        features.append(predict_mean(opinion.comment, mean_std_u_default_2[0], unique=True))

        # 12 defaultDict_discr_hq_strong
        features.append(predict_discr_with_hq_dict(opinion.comment, mean_std_default_hq[0]))
        # 13 defaultDict_hq_discr_strong_unique
        features.append(predict_discr_with_hq_dict(opinion.comment, mean_std_u_default_hq[0], unique=True))
        
        # 14 defaultDict_discr_shq_strong
        features.append(predict_discr_with_hq_dict(opinion.comment, mean_std_default_shq[0]))
        # 15 defaultDict_discr_shq_strong_unique
        features.append(predict_discr_with_hq_dict(opinion.comment, mean_std_u_default_shq[0], unique=True))
        
        # 16 posDict_strong
        features.append(predict_mean(opinion.commentPositive, mean_std_pos[0]))
        # 17 posDict_strong_unique
        features.append(predict_mean(opinion.commentPositive, mean_std_u_pos[0], unique=True))
        
        # 18 posDict2_strong
        features.append(predict_mean(opinion.commentPositive, mean_std_pos_2[0]))
        # 19 posDict2_strong_unique
        features.append(predict_mean(opinion.commentPositive, mean_std_u_pos_2[0], unique=True))
        
        # 20 posDict_discr_hq_strong
        features.append(predict_discr_with_hq_dict(opinion.commentPositive, mean_std_pos_hq[0]))
        # 21 posDict_discr_hq_strong_unique
        features.append(predict_discr_with_hq_dict(opinion.commentPositive, mean_std_u_pos_hq[0], unique=True))
    
        # 22 posDict_discr_shq_strong
        features.append(predict_discr_with_hq_dict(opinion.commentPositive, mean_std_pos_shq[0]))
        # 23 posDict_shq_discr_strong_unique
        features.append(predict_discr_with_hq_dict(opinion.commentPositive, mean_std_u_pos_shq[0], unique=True))
        
        # 24 negDict_strong
        features.append(predict_mean(opinion.commentNegative, mean_std_neg[0]))
        # 25 negDict_strong_unique
        features.append(predict_mean(opinion.commentNegative, mean_std_u_neg[0], unique=True))
        
        # 26 negDict2_strong
        features.append(predict_mean(opinion.commentNegative, mean_std_neg_2[0]))
        # 27 negDict2_strong_unique
        features.append(predict_mean(opinion.commentNegative, mean_std_u_neg_2[0], unique=True))
        
        # 28 negDict_discr_hq_strong
        features.append(predict_discr_with_hq_dict(opinion.commentNegative, mean_std_neg_hq[0]))
        # 29 negDict_discr_hq_strong_unique
        features.append(predict_discr_with_hq_dict(opinion.commentNegative, mean_std_u_neg_hq[0], unique=True))
        
        # 30 negDict_discr_shq_strong
        features.append(predict_discr_with_hq_dict(opinion.commentNegative, mean_std_neg_shq[0]))
        # 31 negDict_discr_shq_strong_unique
        features.append(predict_discr_with_hq_dict(opinion.commentNegative, mean_std_u_neg_shq[0], unique=True))
        
        comment = [opinion.comment,
                   '' if opinion.commentPositive is NaN else opinion.commentPositive,
                   '' if opinion.commentNegative is NaN else opinion.commentNegative]
        comment = ' '.join(comment)

        # 32 commonDict_strong
        features.append(predict_mean(comment, mean_std_common[0]))
        # 33 commonDict_strong_unique
        features.append(predict_mean(comment, mean_std_u_common[0], unique=True))
        
        # 34 commonDict2_strong
        features.append(predict_mean(comment, mean_std_common_2[0]))
        # 35 commonDict2_strong_unique
        features.append(predict_mean(comment, mean_std_u_common_2[0], unique=True))
        
        # 36 commonDict_discr_hq_strong
        features.append(predict_discr_with_hq_dict(comment, mean_std_common_hq[0]))
        # 37 commonDict_discr_hq_strong_unique
        features.append(predict_discr_with_hq_dict(comment, mean_std_u_common_hq[0], unique=True))
        
        # 38 commonDict_discr_shq_strong
        features.append(predict_discr_with_hq_dict(comment, mean_std_common_shq[0]))
        # 39 commonDict_discr_shq_strong_unique
        features.append(predict_discr_with_hq_dict(comment, mean_std_u_common_shq[0], unique=True))
        
        # 40 - 79 Linear model predictions
        features = features + linearModelPredictions[lm_index]
        
        # 80 ... vectors
        features = features + w2v[lm_index]
        
        for i in xrange(40, 60):
            features[i] = int(features[i])
        lm_index += 1
        pool.append(features)
    return pool

In [ ]:
feature_names = [
    "sku",
    "brandId",
    "cat1",
    "cat2",
    "comment",
    "positiveComment",
    "negativeComment",
    "description",
    "defaultDict_strong",
    "defaultDict_strong_unique",
    "defaultDict2_strong",
    "defaultDict2_strong_unique",
    "defaultDict_discr_hq_strong",
    "defaultDict_discr_hq_strong_unique",
    "defaultDict_discr_shq_strong",
    "defaultDict_discr_shq_strong_unique",
    "posDict_strong",
    "posDict_strong_unique",
    "posDict2_strong",
    "posDict2_strong_unique",
    "posDict_discr_hq_strong",
    "posDict_discr_hq_strong_unique",
    "posDict_discr_shq_strong",
    "posDict_discr_shq_strong_unique",
    "negDict_strong",
    "negDict_strong_unique",
    "negDict2_strong",
    "negDict2_strong_unique",
    "negDict_discr_hq_strong",
    "negDict_discr_hq_strong_unique",
    "negDict_discr_shq_strong",
    "negDict_discr_shq_strong_unique",
    "commonDict_strong",
    "commonDict_strong_unique",
    "commonDict2_strong",
    "commonDict2_strong_unique",
    "commonDict_discr_hq_strong",
    "commonDict_discr_hq_strong_unique",
    "commonDict_discr_shq_strong",
    "commonDict_discr_shq_strong_unique",
    
    'defaultMeanLR',
    'defaultNormalizedLR',
    'defaultMeanSVMC',
    'defaultNormalizedSVMC',
    'positiveMeanLR',
    'positiveNormalizedLR',
    'positiveMeanSVMC',
    'positiveNormalizedSVMC',
    'negativeMeanLR',
    'negativeNormalizedLR',
    'negativeMeanSVMC',
    'negativeNormalizedSVMC',
    'commonMeanLR',
    'commonNormalizedLR',
    'commonMeanSVMC',
    'commonNormalizedSVMC',
    'concatMeanLR',
    'concatNormalizedLR',
    'concatMeanSVMC',
    'concatNormalizedSVMC',

    'concatMeanSVMR',
    'concatNormalizedSVMR',
    'concatMeanSGD',
    'concatNormalizedSGD',
    'commonMeanSVMR',
    'commonNormalizedSVMR',
    'commonMeanSGD',
    'commonNormalizedSGD',
    'negativeMeanSVMR',
    'negativeNormalizedSVMR',
    'negativeMeanSGD',
    'negativeNormalizedSGD',
    'positiveMeanSVMR',
    'positiveNormalizedSVMR',
    'positiveMeanSGD',
    'positiveNormalizedSGD',
    'defaultMeanSVMR',
    'defaultNormalizedSVMR',
    'defaultMeanSGD',
    'defaultNormalizedSGD'
]
vector_features = ['w2v_' + str(i) for i in xrange(400)]
feature_names = feature_names + vector_features
cat_features = [0,1,2,3,4,5,6,7, 12,13,14,15, 20,21,22,23, 28,29,30,31, 36,37,38,39] + range(40,60)

In [ ]:
features = get_features(data, testLM.tolist(), vectorsTestMean.tolist())

In [ ]:
pool = Pool(features, cat_features=cat_features, feature_names=feature_names)

# предсказания

In [ ]:
prediction = catBoost.predict(pool)

In [ ]:
RMSE = np.sqrt(np.mean(np.square(prediction - data.reting)))

In [ ]:
RMSE